1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 문서 임베딩 후 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

spltr=RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200, # 문서를 자를 때 어느정도 겹치게 할 지 (앞뒤 문맥을 같이 전달해주기 위함)
)

loader=Docx2txtLoader('./tax.docx')
doc_list=loader.load_and_split(text_splitter=spltr)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 임베딩 과정에도 환경변수 필요하므로 불러와줘야함

embedding=OpenAIEmbeddings(model='text-embedding-3-large')

In [3]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name='tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database=PineconeVectorStore.from_documents(doc_list,embedding,index_name=index_name)

c:\Users\User\Desktop\rag-practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query='연봉 5000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs=database.similarity_search(query, k=3)

In [5]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-4o')

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 최고의 한국 소득세 전문가입니다."
     "Context를 참고해서 사용자의 질문에 답변해주세요."),
    ("user",
     "문서(context):\n{context}\n\n"
     "질문(question): {question}\n\n"
     "답변:")
])

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = database.as_retriever(search_kwargs={"k": 3})

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
ai_msg=rag_chain.invoke(query)
ai_msg

'연봉 5000만원인 직장인의 소득세를 정확하게 계산하기 위해서는 근로소득 공제, 각종 세액공제, 소득세율 등을 고려해야 합니다. 한국의 소득세는 과세표준에 따라 누진세율이 적용됩니다. 기본적으로 2023년 기준으로 소득세율은 다음과 같습니다:\n\n- 과세표준 1,200만원 이하: 6%\n- 1,200만원 초과 4,600만원 이하: 15%\n- 4,600만원 초과 8,800만원 이하: 24%\n- 8,800만원 초과 1억5천만원 이하: 35%\n- 1억5천만원 초과 3억원 이하: 38%\n- 3억원 초과 5억원 이하: 40%\n- 5억원 초과: 42%\n\n이를 바탕으로 대략적인 계산을 하면, 연봉 5000만원은 과세표준 4,600만원 초과 8,800만원 이하 구간에 해당합니다. 근로소득세 계산은 다음과 같이 진행됩니다.\n\n1. **근로소득 공제**: 이는 연봉에 따라 자동 적용되는 공제로 과세표준을 낮추는데 사용됩니다.\n2. **과세표준 계산**: 연봉에서 근로소득 공제와 기타 공제를 차감하여 계산됩니다.\n3. **적용 세율에 따라 소득세 계산**: 과세표준에 누진세율을 적용하여 소득세를 계산합니다.\n4. **세액공제 적용**: 소득세에서 각종 세액공제를 반영하여 최종 납부할 세액을 산출합니다.\n\n정확한 소득세 금액을 계산하기 위해서는 상세한 세부사항, 예를 들어 입력할 세액공제 항목이 필요합니다. 기본적인 개념을 참고하고 필요 시 국세청 홈택스나 세무 전문가의 도움을 받아 정확히 계산하시기 바랍니다.'